In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Reding the .csv file 

In [ ]:
df_autoscout24 = pd.read_csv("/kaggle/input/autoscout24-autosphere-scrap-results/deleted-empty-rows.csv")
df_autosphere = pd.read_csv("/kaggle/input/autoscout24-autosphere-scrap-results/dataset.csv")

In [ ]:
df_autoscout24.shape

In [ ]:
df_autosphere.shape

In [ ]:
df_autoscout24 = df_autoscout24.drop(columns=["Unnamed: 0"])

In [ ]:
df_autoscout24["price"].isna().sum()

In [ ]:
df_autoscout24 = df_autoscout24.dropna(subset=["price"])
df_autoscout24.shape

In [ ]:
df_autoscout24.head()

In [ ]:
def count_words(row):
    words = row.split()
    return(len(words))
df_autoscout24["model_name"].apply(count_words).unique()

In [ ]:
df_autoscout24[df_autoscout24["model_name"].str.startswith("M")]["model_name"].apply(count_words)

In [ ]:
filtered_rows = df_autoscout24[df_autoscout24["model_name"].str.startswith("M")].apply(lambda row: count_words(row["model_name"]) == 4 , axis=1)
filtered_indexes = filtered_rows[filtered_rows].index


In [ ]:
df_autoscout24.loc[116]

## Separating the brand and the model

In [ ]:
def get_brand(row):
    words = row.split()
    return words[0]

def get_model_name(row):
    words = row.split()
    if len(words) > 1:
        return " ".join(words[1:])

In [ ]:
df_autoscout24["brand"] = df_autoscout24["model_name"].apply(get_brand)

In [ ]:
df_autoscout24["model"] = df_autoscout24["model_name"].apply(get_model_name)

In [ ]:
df_autoscout24 = df_autoscout24.drop(columns=["model_name"])

## We have to convert kilometers into interger value

In [ ]:
df_autoscout24["kilometers"].isna().sum()

In [ ]:
def get_kilometers(row):
    words = str(row).split()
    if "".join(words[:len(words)-1]) == "-" or not "".join(words[:len(words)-1]):
        return None
    return int("".join(words[:len(words)-1]))
df_autoscout24["kilometers"] = df_autoscout24["kilometers"].apply(get_kilometers)

## Converting the price to integer too

In [ ]:
import re
def longest_series_of_digits(input_string):
    input_string = input_string.replace(" ", "")
    digit_series = re.findall(r'\d+', input_string)
    if not digit_series:
        return None
    concatenated_digits = ''.join(digit_series)
    longest_digit_series = max(re.findall(r'\d+', concatenated_digits), key=len)
    return int(longest_digit_series)

df_autoscout24["price"] = df_autoscout24["price"].apply(longest_series_of_digits)

## Working on price_judgement column

In [ ]:
df_autoscout24[df_autoscout24["price_judgement"].isna()] = "Pas d'information"

In [ ]:
df_autoscout24["price_judgement"].unique()

In [ ]:
print(len(df_autoscout24[df_autoscout24["price_judgement"] == "Pas d'information"]))
print(len(df_autoscout24[df_autoscout24["price_judgement"] == "Bon prix"]))
print(len(df_autoscout24[df_autoscout24["price_judgement"] == "Très bon prix"]))
print(len(df_autoscout24[df_autoscout24["price_judgement"] == "Prix correct"]))

## Dealing with the transmission

In [ ]:
df_autoscout24[df_autoscout24["transmission"] == "- Boîte"] = "Pas d'information"

In [ ]:
df_autoscout24["transmission"].unique()

In [ ]:
print(len(df_autoscout24[df_autoscout24["transmission"] == "Pas d'information"]))
print(len(df_autoscout24[df_autoscout24["transmission"] == "Boîte manuelle"]))
print(len(df_autoscout24[df_autoscout24["transmission"] == "Boîte automatique"]))
print(len(df_autoscout24[df_autoscout24["transmission"] == "Semi-automatique"]))

In [ ]:
df_autoscout24["transmission"] = df_autoscout24["transmission"].fillna("Pas d'information")

## Dealing with the fuel

In [ ]:
df_autoscout24["fuel"].unique()

In [ ]:
df_autoscout24["fuel"] = df_autoscout24["fuel"].fillna("Pas d'information")

In [ ]:
df_autoscout24.loc[df_autoscout24["fuel"] == "- Carburant", "fuel"] = "Pas d'information"

In [ ]:
df_autoscout24.isna().sum()

In [ ]:
df_autoscout24[df_autoscout24["release_date"].isna()]

In [ ]:
df_autoscout24["release_date"].unique()

In [ ]:
def extract_year(row):
    if pd.isna(row) or row == "Pas d'information" or row == "- (Année)":
        return None
    words = str(row).split("/")
    if len(words) == 2:
        return words[1]
    if len(words) == 1:
        return words[0]  # Just return the year directly

df_autoscout24["release_year"] = df_autoscout24["release_date"].apply(extract_year)

## Dealing with the horsepower

In [ ]:
def extract_power(row):
    if pd.isna(row) or row == "Pas d'information":
        return None
    if "CH" in str(row):
        words = str(row).split()
        if len(words) > 1:
            if words[len(words) - 2][1:].isdigit():
                return int(words[len(words) - 2][1:])
            else:
                return None
df_autoscout24["horse_power"] = df_autoscout24['power'].apply(extract_power)


In [ ]:
def extract_transmission(row):
    words = row.split()
    if (row == "Pas d'information"):
        return None
    if len(words) == 2:
        return words[1]
    else:
        return words[0]
df_autoscout24['transmission'] = df_autoscout24['transmission'].apply(extract_transmission)
# df_autoscout24['transmission'].unique()

In [ ]:
df_autoscout24 = df_autoscout24.drop(columns=["release_date"])

## Dealing with price judgement

### I prefer to let the price judgement with pas d'information as a value like they are 

In [ ]:
df_autoscout24 = df_autoscout24.drop(columns=["power"])

In [ ]:
df_autoscout24.head()

## We have very few rows where the fuel is not mentionned, we delete them   

In [ ]:
df_autosphere = df_autosphere[df_autosphere["fuel"] != "/"]

# Extracting just the fuel type 

In [ ]:
def extract_fuel(row):
    if row and row[len(row) - 1] == "/":
        return row[:len(row)-2]
df_autosphere["fuel"] = df_autosphere["fuel"].apply(extract_fuel)


# Descovering the gearbox column

In [ ]:
df_autosphere["gearbox"].isna().sum()

# Extracting the power from the descrption 

In [ ]:
import re

def extract_power(row):
    words = str(row).split()
    pattern = r'\d+ch$'
    for word in words:
        if re.match(pattern,word):
            return int(word[:len(word)-2])
    return None
df_autosphere["power"] = df_autosphere["description"].apply(extract_power)

# Dealing with the price 

In [ ]:
def extract_price(row):
    return int(row[:len(row)-2].replace(" ", ""))
df_autosphere["price"] = df_autosphere["price"].apply(extract_price)

# Dealing with the year

In [ ]:
def extract_price(row):
    pattern = r'\d+'
    digits = re.findall(pattern, row)
    return "".join(digits)

df_autosphere["year"] = df_autosphere["year"].apply(extract_price)

# Dealing with kilometers

In [ ]:
def extract_kilometers(row):
    if len(row) > 3:
        return int(row[:len(row)-3].replace(" ", ""))
    return None
df_autosphere["km"] = df_autosphere["km"].apply(extract_kilometers)

In [ ]:
df_autoscout24.head()

# Starting with the autosphere dataset 

In [ ]:
df_autosphere.shape[0] - df_autosphere["auto_clim"].sum()

In [ ]:
df_autosphere.head()

In [ ]:
print(df_autosphere.columns)
print(df_autoscout24.columns)

In [ ]:
def brand_to_capital(row):
    return row.upper()
df_autoscout24["brand"] = df_autoscout24["brand"].apply(brand_to_capital)

In [ ]:
df_autoscout24["transmission"].unique()

In [ ]:
df_autosphere["gearbox"].unique()

In [ ]:
def adjust_gearbox(row):
    if row == "automatique":
        return "Automatique"
    if row == "manuelle":
        return "Manuelle"
    return row

df_autoscout24["transmission"] = df_autoscout24["transmission"].apply(adjust_gearbox)

In [ ]:
df_autoscout24['fuel'].unique()

In [ ]:
df_autosphere["fuel"].unique()

In [ ]:
same_names_fuels = []
different_name = []
for x in df_autoscout24['fuel'].unique():
    for y in df_autosphere["fuel"].unique():
        if x == y:
            same_names_fuels.append(x)
for x in df_autoscout24['fuel'].unique():
    if x not in same_names_fuels:
        different_name.append(x)
for x in df_autosphere['fuel'].unique():
    if x not in same_names_fuels:
        different_name.append(x)
print(same_names_fuels)
print(different_name)

## Can let it like this 

In [ ]:
df_autoscout24_to_fusion = pd.DataFrame()
df_autoscout24_to_fusion["mark"] = df_autoscout24["brand"]
df_autoscout24_to_fusion["model"] = df_autoscout24["model"]
df_autoscout24_to_fusion["description"] = df_autoscout24["extended_name"]
df_autoscout24_to_fusion["fuel"] = df_autoscout24["fuel"]
df_autoscout24_to_fusion["km"] = df_autoscout24["kilometers"]
df_autoscout24_to_fusion["year"] = df_autoscout24["release_year"]
df_autoscout24_to_fusion["gearbox"] = df_autoscout24["transmission"]
df_autoscout24_to_fusion["price"] = df_autoscout24["price"]
df_autoscout24_to_fusion["power"] = df_autoscout24["horse_power"]
df_autoscout24_to_fusion["isautoscout"] = 1
df_autoscout24_to_fusion.isna().sum()

## Imputing the power column with the mean

In [ ]:
column_mean = df_autoscout24_to_fusion['power'].mean()
df_autoscout24_to_fusion['power'] = df_autoscout24_to_fusion['power'].fillna(column_mean)

## Imputing the km column with the median

In [ ]:
def dealing_with_km(row):
    if row == "Pas d'information":
        return None
    return float(row)
df_autoscout24_to_fusion["km"] = df_autoscout24_to_fusion["km"].apply(dealing_with_km)

In [ ]:
column_median = df_autoscout24_to_fusion['km'].median()
df_autoscout24_to_fusion['km'] = df_autoscout24_to_fusion['km'].fillna(column_median)

## Deleting the rows with no description, gearbox and year

In [ ]:
df_autoscout24_to_fusion = df_autoscout24_to_fusion.dropna(axis=0)

In [ ]:
df_autoscout24_to_fusion["price"] = df_autoscout24_to_fusion["price"].apply(dealing_with_km)

In [ ]:
df_autoscout24_to_fusion.dtypes

In [ ]:
df_autosphere_to_fusion = df_autosphere.drop(columns=['isofix', 'led', 'cruise_control', 'bluetooth', 'auto_clim', 'rear_cam','rear_rad'])
df_autosphere_to_fusion["isautoscout"] = 0
df_autosphere_to_fusion.dtypes

In [ ]:
power_mean = df_autosphere_to_fusion["power"].mean()
df_autosphere_to_fusion["power"] = df_autosphere_to_fusion["power"].fillna(power_mean)

In [ ]:
df_autosphere_to_fusion["km"] = df_autosphere_to_fusion["km"].astype("float")
df_autosphere_to_fusion["price"] = df_autosphere_to_fusion["price"].astype("float")

In [ ]:
df_autosphere_to_fusion.dtypes

In [ ]:
fused_df = pd.concat([df_autosphere_to_fusion, df_autoscout24_to_fusion], axis=0, ignore_index=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
non_numeric_columns = fused_df.select_dtypes(exclude=['number']).columns.tolist()
label_encoder = LabelEncoder()
for col in non_numeric_columns:
    fused_df[col] = label_encoder.fit_transform(fused_df[col])

In [ ]:
labled_autoscout24 = fused_df[fused_df["isautoscout"] == 1]
labled_autoshpere = fused_df[fused_df["isautoscout"] == 0]

In [ ]:
from sklearn.model_selection import train_test_split
def options_to_string(row):
    return str(row["isofix"]) + str(row["led"]) + str(row["cruise_control"]) + str(row["bluetooth"]) + str(row["auto_clim"]) + str(row["rear_cam"]) + str(row["rear_rad"])

y = df_autosphere.apply(options_to_string, axis=1)
X = labled_autoshpere

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
predicted_options_autoscout24 = rf_classifier.predict(labled_autoscout24)

In [ ]:
df_autoscout24_to_fusion["options"] = predicted_options_autoscout24

In [ ]:
# ['isofix', 'led', 'cruise_control', 'bluetooth', 'auto_clim', 'rear_cam','rear_rad']
def get_options_from_string(row):
    row["isofix"] = int(row["options"][0])
    row["led"] = int(row["options"][1])
    row["cruise_control"] = int(row["options"][2])
    row["bluetooth"] = int(row["options"][3])
    row["auto_clim"] = int(row["options"][4])
    row["rear_cam"] = int(row["options"][5])
    row["rear_rad"] = int(row["options"][6])
    return row
    
df_autoscout24_to_fusion = df_autoscout24_to_fusion.apply(get_options_from_string,axis=1)

In [ ]:
df_autoscout24_to_fusion

In [ ]:
 y

In [ ]:
df_autosphere_to_fusion["options"] = y
df_autosphere_to_fusion = df_autosphere_to_fusion.apply(get_options_from_string,axis=1)

In [ ]:
df_autosphere_to_fusion

In [ ]:
fused_df = pd.concat([df_autosphere_to_fusion, df_autoscout24_to_fusion], axis=0, ignore_index=True)

In [ ]:
fused_df = fused_df.drop(columns=["options","isautoscout"])

In [ ]:
fused_df.to_csv("clean_fused_dataset.csv")